## Simulate sampling the returned time-series of concentrations
<br>
  
#### Syntax
`CalcPars = CNSamplingFxn(RunPars, CosmoPars, HeliumPars, CalcPars)` <br>
#### Input 
`RunPars` : dictionary of terms consistent for the whole run of the model.<br>
`CosmoPars` : dictionary of CRN parameters <br>
`HeliumPars` : dictionary of terms relevant for He-3 production and diffusion . <br>
`CalcPars` : dictionary of concentrations from the simulations <br>
#### Variables Used
##### Run Pars
`total_time` : total time for the model (yrs) <br>
`s` : total number of scenarios <br>
`sample_no` : total number of samples to be "collected" from random years <br>
##### Cosmo Pars
`SLHL_Be10` : sea level high latitude Be-10 spall production rate (at g-1 yr-1) <br>
`SLHL_C14` : sea level high latitude C-14 spall production rate (at g-1 yr-1)   <br>
`SLHL_He3` : sea level high latitude He-3 spall production rate (at g-1 yr-1) <br> 
`scaling_factor` : CN scaling parameter (e.g. Stone 20000) <br> 
`rock_rho` = density of the rock (g cm-3) <br> 
`Be10Decay` = Be-10 decay constant (yr-1) <br> 
`L` : CN attenuation lengthscale (g cm-2) <br> 
##### Calc Pars
`crnSurf` : surface CRN nuclide concentrations size s * total_time (at g-1) <br> 
`HeSurf` : matrix of surface nuclide concentration for He-3 size s * total_time (at g-1) <br> 
#### Output
##### Calc Pars
`Sample Matrix` : matrix containing sampled concentrations, size (sample_no * 4 * s) <br>
`Sampling Years` : years from which the samples are collected <br>
`Measurements Matrix` = matrix containing measurements (14C/10Be, Retention, & Erosion Rate), size 6 * scenarios <br> 
`Measured Erosion Rates` = vector size s of the apparent erosion rates calculated from Be-10 <br>
#### Notes
**Date of Creation:** 7. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [1]:
def CNSamplingFxn(RunPars, CosmoPars, HeliumPars, CalcPars):
    # read-in relevant variables   
    # RunPars
    total_time = RunPars['total_time']
    s = RunPars['scenarios']
    sample_no = RunPars['sample_no']
    #CosmoPars
    SLHL_Be10 = CosmoPars['SLHL_Be10']
    SLHL_C14 = CosmoPars['SLHL_C14']
    SLHL_He3 = CosmoPars['SLHL_He3']
    scaling_factor = CosmoPars['scaling_factor']
    rock_rho = CosmoPars['rock_rho']
    Be10Decay = CosmoPars['Be10 Decay']
    L1 = CosmoPars['CN_lambda']    
    L2 = CosmoPars['L3 Be']
    muonP0 = CosmoPars['B_Be']
    spallperc = SLHL_Be10 / (SLHL_Be10 + muonP0)
    muonperc = muonP0 / (SLHL_Be10 + muonP0)
    
    #CalcPars
    crnSurf = CalcPars['CRN Surface Conc.']
    HeSurf = CalcPars['He Surf Matrix']
    # scale production rates
    P0Be10 = SLHL_Be10 * scaling_factor
    P0C14 = SLHL_C14 * scaling_factor
    P0He3 = SLHL_He3 * scaling_factor
    # determine the sampled years
    samp_years = np.sort(np.random.choice(np.arange(int(total_time*0.1), total_time),sample_no))
    # open up empty matrices to store samples and calculations
    samp_mat = np.empty((sample_no, 4, s))
    meas_mat = np.empty((6, s))
    # loop through sample assignments
    for i in range(sample_no):
        samp_mat[i,:3,:] = crnSurf[:,samp_years[i],:]
        samp_mat[i,3,:] = HeSurf[samp_years[i],:]
    # make the calculations for every year
    for j in range(s):
        meas_mat[:4,j] = np.mean(samp_mat[:,:,j], axis = 0)
        meas_mat[4,j] = np.divide(meas_mat[3,j],meas_mat[1,j]) * (P0Be10 / P0He3)
        meas_mat[5,j] = ErosionRateCalc(meas_mat[1,j], P0Be10, spallperc, muonperc, Be10Decay, L1, L2, rock_rho)
    # assign the calculations to the calc pars dictionary
    CalcPars['Sample Matrix'] = samp_mat,
    CalcPars['Sampling Years'] = samp_years,
    CalcPars['Measurements Matrix'] = meas_mat
    CalcPars['Measured Erosion Rates'] = meas_mat[5,:]
    
    return CalcPars
    